# 네이버 영화정보 크롤링
- top_movie_data에 들어있는 영화들에 대한 정보 크롤링
- 'https://movie.naver.com/' 여기서 영화검색을 입력하고 그 영화의 url상 고유코드번호를 먼저 수집한다
    - ex) 
    - 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=69783'
- 수집한 고유코드를 영화제목에 대응시켜서 dataframe을 만든다
- 영화정보 url에 영화 고유코드만 바꿔가면서 scrapy로 크롤링한다
- 영화 평점리뷰도 페이지를 url로 바꿔서 크롤링 할 수 있다(마지막의 page 바꾸면 됨)
    - https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=44885&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=4
- 배우 
    - https://movie.naver.com/movie/bi/mi/detail.nhn?code=44885
    
- 크롤링 과정
    - top_movie_data에서 영화제목 컬럼만 가져와서, 셀레니움으로 영화제목을 검색란 input에 넣고 엔터친다음에 
    - webdriver.current_url로 현재 url의 영화제목 고유코드를 dataframe에 모은다
    - 이 dataframe을 완성한 이후에는 
    - `https://movie.naver.com/movie/bi/mi/basic.nhn?code={}'.format()` 이거로 영화제목의 고유코드로 scrapy를 이용해서 정보 크롤링
    
- movie_title_list[718] : '독전 - 익스텐디드 컷' -> 네이버 영화 검색결과가 없어서 그냥 '독전'으로 찾아서 넣음

- 평론가평점
- 주연배우  
    - 주연배우는 앞쪽부터 3명
- 러닝타임
- spider클래스의 parse함수에서 for문을 통해 id 넣기(index로)
- 다 수집하면 id 순서로 데이터프레임을 재정렬하고 기존의 데이터프레임과 concat한다
- 감독은, 나중에 혹시나 이름이 같은 다른 영화를 클로링했는지 확인하기 위해 크롤링한다
- 정확히 내가 찾는 영화를 가져왔는지 나중에 확인하기 위해, 영화제목과 개봉일시도 일단 크롤링 한다

### 네이버 영화 고유코드 수집

In [160]:
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
from selenium import webdriver
import re
import matplotlib as mpl
from matplotlib import rc
import missingno as msno
import warnings
from scrapy.http import TextResponse
warnings.filterwarnings('ignore')

# matplotlib 한글 사용을 위한 코드
rc('font', family='AppleGothic')

In [10]:
top_movie_data = pd.read_csv('top_movie_data.csv')
# top_movie_data.head(2)

In [11]:
top_movie_data = top_movie_data.rename(columns={'Unnamed: 0':'id'})

In [12]:
top_movie_data.head(2)

,id,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,영화구분
0,0,어벤져스: 에이지 오브 울트론,조스 웨던,NaN,월트디즈니컴퍼니코리아(주),월트디즈니컴퍼니코리아(주),2015-04-23,개봉영화,장편,미국,1843,88582586366,10494499,25001906266,2823388,액션,12세이상관람가,일반영화
1,1,국제시장,윤제균,"(주)제이케이필름,씨제이이앤엠(주)",NaN,씨제이이앤엠(주),2014-12-17,개봉영화,장편,한국,966,69823893034,8911437,16331193634,2033863,드라마,12세이상관람가,일반영화


In [31]:
movie_title_list = list(top_movie_data['영화명'])
len(movie_title_list)

928

In [32]:
url = 'https://movie.naver.com/'
driver = webdriver.Chrome()
driver.get(url)

In [33]:
code_datas = []
title_datas = []
comeout_date_datas = []
expert_ratings_datas = []
starring_actors_datas = []
running_times_datas = []
directors_datas = []

In [34]:
def check_response(selector, start_time):
    if time.time() - start_time > 5:
        raise Exception
        return
    try:
        element = driver.find_element_by_css_selector(selector)
    except Exception:
        return check_response(selector, start_time)
    pass

In [151]:
for i in range(737, len(movie_title_list)):
    input_element = driver.find_element_by_css_selector('#ipt_tx_srch')
    input_element.send_keys(movie_title_list[i])
    check_response('#jAutoComplate', time.time())
    
    
    try:
        driver.find_element_by_css_selector('#jAutoMV > ul > li > a').click()
    except Exception:
        driver.find_element_by_css_selector('#jAutoMV > ul > li:nth-child(1) > a').click()
        
    check_response('#content > div.article > div.mv_info_area > div.mv_info', time.time())
    
        
    splited_url = driver.current_url.split('code=')
    code_data = {'N_movie_code' : splited_url[1]}
    
    title = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > h3 > a').text
    title_data = {'title': title}
    
    comeout_date_element = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > strong')
    try:          
        date = comeout_date_element.text.split(',')
        if len(date) == 1:
            date = date[0].strip()
        else:
            date = date[1].strip()     
    except Exception:
        date = '-'
        
    comeout_date_data = {'comeout_date': date}
        
    try:    
        expert_element = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > div.main_score > div:nth-child(2) > div > a > div')
        expert_rating = expert_element.text
    except Exception:
        expert_rating = '-'
    expert_rating_data = {'expert_rating': expert_rating}
    
    try:
        starring_actors_element = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(6) > p')
        starring_actors = starring_actors_element.text
    except Exception:
        starring_actors = '-'
    starring_actors_data = {'starring_actors': starring_actors}
    
    try:
        running_times_element = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(3)')
        running_times = running_times_element.text
    except Exception:
        running_times = '-'
    running_times_data = {'running_times': running_times}
    
    try:
        directors_element = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(4) > p')
        directors = directors_element.text
    except Exception:
        directors = '-'
    directors_data = {'directors': directors}
        
    
    code_datas.append(code_data)
    title_datas.append(title_data)
    comeout_date_datas.append(comeout_date_data)
    expert_ratings_datas.append(expert_rating_data)
    starring_actors_datas.append(starring_actors_data)
    running_times_datas.append(running_times_data)
    directors_datas.append(directors_data)
    
    

In [15]:
# element = driver.find_element_by_css_selector('#jAutoMV > ul > li:nth-child(1) > a').click()
# elements[0].text
# elements[1].text
# element.text.split(',')[0].strip()
# element.text
# element
        
        

In [152]:
len(directors_datas)

928

In [21]:
# starring_actors_datas

In [302]:
len(datas)

928

In [144]:
movie_title_list[736]

'나츠메 우인장: 세상과 연을 맺다'

In [149]:
directors_datas[736]

{'directors': '오오모리 타카히로, 이토 히데키'}

In [147]:
running_times_datas[736]

{'running_times': '104분'}

In [177]:
# datas[927]

In [175]:
# df.at[3, 'N_movie_code'] = '67786' 수동으로 영화코드 수정 (필요한 경우)

In [217]:
# df.to_csv('code_added_df.csv', mode='w')

In [ ]:
# code_datas.append({'N_movie_code' : '-'})
# title_datas.append({'title': '-'})
# comeout_date_datas.append({'comeout_date': '-'})
# expert_ratings_datas.append({'expert_rating': '-'})
# starring_actors_datas.append({'starring_actors': '-'})
# running_times_datas.append({'running_times': '-'})
# directors_datas.append({'directors': '-'})

In [137]:
del code_datas[len(code_datas) - 1]
del title_datas[len(title_datas) - 1]
del comeout_date_datas[len(comeout_date_datas) - 1]
del expert_ratings_datas[len(expert_ratings_datas) - 1]
del starring_actors_datas[len(starring_actors_datas) - 1]
del running_times_datas[len(running_times_datas) - 1]
del directors_datas[len(directors_datas) - 1]

In [115]:
len(directors_datas)

718

In [102]:
def once():    
    try:
        driver.find_element_by_css_selector('#jAutoMV > ul > li > a').click()
    except Exception:
        driver.find_element_by_css_selector('#jAutoMV > ul > li:nth-child(1) > a').click()
        
    check_response('#content > div.article > div.mv_info_area > div.mv_info', time.time())
    
        
    splited_url = driver.current_url.split('code=')
    code_data = {'N_movie_code' : splited_url[1]}
    
    title = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > h3 > a').text
    title_data = {'title': title}
    
    comeout_date_element = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > strong')
    try:          
        date = comeout_date_element.text.split(',')
        if len(date) == 1:
            date = date[0].strip()
        else:
            date = date[1].strip()     
    except Exception:
        date = '-'
        
    comeout_date_data = {'comeout_date': date}
        
    try:    
        expert_element = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > div.main_score > div:nth-child(2) > div > a > div')
        expert_rating = expert_element.text
    except Exception:
        expert_rating = '-'
    expert_rating_data = {'expert_rating': expert_rating}
    
    try:
        starring_actors_element = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(6) > p')
        starring_actors = starring_actors_element.text
    except Exception:
        starring_actors = '-'
    starring_actors_data = {'starring_actors': starring_actors}
    
    try:
        running_times_element = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(3)')
        running_times = running_times_element.text
    except Exception:
        running_times = '-'
    running_times_data = {'running_times': running_times}
    
    try:
        directors_element = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(4) > p')
        directors = directors_element.text
    except Exception:
        directors = '-'
    directors_data = {'directors': directors}
        
    
    code_datas.append(code_data)
    title_datas.append(title_data)
    comeout_date_datas.append(comeout_date_data)
    expert_ratings_datas.append(expert_rating_data)
    starring_actors_datas.append(starring_actors_data)
    running_times_datas.append(running_times_data)
    directors_datas.append(directors_data)
    

In [145]:
once()

In [153]:
df1 = pd.DataFrame(code_datas)
df2 = pd.DataFrame(title_datas)
df3 = pd.DataFrame(comeout_date_datas)
df4 = pd.DataFrame(expert_ratings_datas)
df5 = pd.DataFrame(starring_actors_datas)
df6 = pd.DataFrame(running_times_datas)
df7 = pd.DataFrame(directors_datas)

In [156]:
len(df1), len(df2), len(df3), len(df4), len(df5), len(df6), len(df7)

(928, 928, 928, 928, 928, 928, 928)

In [157]:
new_df = pd.concat([df1, df2, df3, df4, df5, df6, df7], axis=1)
new_df

,N_movie_code,title,comeout_date,expert_rating,starring_actors,running_times,directors
0,98438,어벤져스: 에이지 오브 울트론,2015,6.92,"로버트 다우니 주니어(토니 스타크 / 아이언맨), 크리스 헴스워스(토르), 마크 러...",141분,조스 웨던
1,102875,국제시장,2014,5.81,"황정민(덕수), 김윤진(영자), 오달수(달구)",126분,윤제균
2,114249,킹스맨 : 시크릿 에이전트,2015,7.58,"콜린 퍼스(해리), 태런 에저튼(에그시), 사무엘 L. 잭슨(발렌타인)",128분,매튜 본
3,191646,쥬라기 월드: 도미니언,2021,-,"크리스 프랫, 브라이스 달라스 하워드",2021 .06.10 개봉,콜린 트레보로우
4,123596,조선명탐정 : 사라진 놉의 딸,2014,5.85,"김명민(명탐정 김민), 오달수(서필), 이연희(히사코)",125분,김석윤
...,...,...,...,...,...,...,...
923,179398,메기,2018,6.83,"이주영(여윤영), 문소리(이경진), 구교환(이성원)",89분,이옥섭
924,174748,프란치스코 교황: 맨 오브 히스 워드,2018,6.33,프란치스코(본인),96분,빔 벤더스
925,186367,빅 위시,Salma's Big Wish,6.00,[국내] 전체 관람가,79분,카를로스 구티에레스 메드라노
926,187787,너를 만난 여름,My Best Summer,5.50,"진비우(위화이), 하람두(겅겅)",109분,장적사


In [158]:
new_df.to_csv('except_audience_rating_df.csv', mode='w')

In [311]:
movie_code_list = list(new_df['N_movie_code'])

In [312]:
len(movie_code_list)

928

### request & reponse : 관람객 평점 크롤링

- 관람객 평점(audience rating):
    -   `https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhncode=154285&type=after&onlyActualPointYn=Y&onlySpoilerPointYn=N&order=newest&page=9999`
    - order는 최신순으로 정렬
    - onlyActualPointYn 쿼리는 관람객 평점만 볼지 말지 정하는거
    - page 쿼리는 엄청 큰 숫자 쓰면 가장 마지막 페이지로 간다
- 셀레니움으로 맨 마지막 페이지에 도착한 상태로 시작하면, 먼저 현재 페이지를 변수에 담는다
- 그 다음부턴 현재페이지 - 1 한 페이지를 현재페이지 변수에 담아서 url에 사용하고
- 그 url페이지의 평점을 계속 더하기 한다
- 더하기 하면서 날짜를 확인하고 개봉일자로부터 1주일이 지나면 평점 더하기를 중단한다


In [401]:
#pagerTagAnchor364
import datetime

In [422]:
audience_rating = []
rating_audiences = []

In [423]:
for i in range(0, len(movie_code_list)):
    
    # 먼저, 평점의 가장 마지막 페이지에 해당하는 url의 document구조를 가져온다
    code = int(movie_code_list[i])
    num = 99999
    url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={}&type=after&onlyActualPointYn=Y&onlySpoilerPointYn=N&order=newest&page={}'.format(code, num)
    
    # 현재 페이지 알아내기
    try:
        req = requests.get(url)
        response = TextResponse(req.url, body=req.text, encoding="utf-8")
        number_list = response.xpath('/html/body/div/div/div[6]/div/a/span/text()').extract()
        last_number = number_list[len(number_list) - 1]

        cur_page = int(last_number.replace(',', ''))
        cur_page

        # 평가의 마지막 페이지 마지막 관객의 평점올린 날짜를 가져와서 7주일 이후의 날짜(limit_date) 만들기
        time_elements = response.xpath('/html/body/div/div/div[5]/ul/li/div/dl/dt/em[2]/text()')
        start_date = time_elements[len(time_elements) - 1].extract().split(' ')[0]
        start_date = start_date.split(" ")[0].split('.')
        limit_date = datetime.datetime(int(start_date[0]), int(start_date[1]), int(start_date[2])) + datetime.timedelta(weeks=1)

        audience_number = 0
        total_rating = 0
        flag = 0

        # 여기서부터 정확한 페이지로 크롤링 시작
        for page in range(cur_page, 0, -1):
            if flag == 1:
                break

            url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={}&type=after&onlyActualPointYn=Y&onlySpoilerPointYn=N&order=newest&page={}'.format(code, page)
            req = requests.get(url)
            response = TextResponse(req.url, body=req.text, encoding="utf-8")

            rating_elements = response.xpath('/html/body/div/div/div[5]/ul/li')

            # 평가의 해당 페이지에서, 각 평점 엘리먼트 하나마다 날짜를 확인하면서 limit을 넘겼는지 체크한다
            for n in range(len(rating_elements), 0, -1):
                star = response.xpath('/html/body/div/div/div[5]/ul/li[{}]/div[1]/em/text()'.format(n)).extract()[0]
                cur_date = response.xpath('/html/body/div/div/div[5]/ul/li[{}]/div[2]/dl/dt/em[2]/text()'.format(n)).extract()[0].split(' ')[0].split('.')

                check_limit = limit_date - datetime.datetime(int(cur_date[0]), int(cur_date[1]), int(cur_date[2]))
                if check_limit.days < 0:
                    flag = 1
                    break

                total_rating = total_rating + int(star)
                audience_number += 1

        rating = {'audience_rating': total_rating/audience_number}
        audience = {'rating_audiences': audience_number}

        audience_rating.append(rating)
        rating_audiences.append(audience)
        
    except Exception:
        rating = {'audience_rating': '-'}
        audience = {'rating_audiences': '-'}
        audience_rating.append(rating)
        rating_audiences.append(audience)
    

In [424]:
len(rating_audiences)

928

In [425]:
len(rating_audiences)

928

In [426]:
df8 = pd.DataFrame(audience_rating)
df9 = pd.DataFrame(rating_audiences)

In [427]:
new_df2 = pd.concat([new_df, df8, df9], axis=1)

In [431]:
new_df2.to_csv('result_df.csv', mode='w')

In [455]:
a = pd.read_csv('result_df.csv')

In [456]:
# a[a['audience_rating'] == '-']

In [468]:
def setting_nan(x):
    if x == '-':
        return float('NaN')
    else:
        return x

In [471]:
a['expert_rating'] = a['expert_rating'].apply(lambda x : float('NaN') if x == '-' else x)

In [490]:
a['directors'] = a['directors'].apply(lambda x : float('NaN') if x == '-' else x)

In [492]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928 entries, 0 to 927
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        928 non-null    int64 
 1   N_movie_code      928 non-null    int64 
 2   title             928 non-null    object
 3   comeout_date      928 non-null    object
 4   expert_rating     814 non-null    object
 5   starring_actors   927 non-null    object
 6   running_times     922 non-null    object
 7   directors         928 non-null    object
 8   audience_rating   876 non-null    object
 9   rating_audiences  928 non-null    object
dtypes: int64(2), object(8)
memory usage: 72.6+ KB


In [493]:
a.to_csv('result_df_NaN.csv', mode='w')

In [421]:
code = 191646
num = 99999
url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={}&type=after&onlyActualPointYn=Y&onlySpoilerPointYn=N&order=newest&page={}'.format(code, num)
req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
number_list = response.xpath('/html/body/div/div/div[6]/div/a/span/text()').extract()
last_number = number_list[len(number_list) - 1]

cur_page = int(last_number.replace(',', ''))
cur_page



IndexError: list index out of range

In [415]:
# start_date = response.xpath('/html/body/div/div/div[5]/ul/li[5]/div[2]/dl/dt/em[2]/text()').extract()
# start_date = start_date[0].split(" ")[0].split('.')
# start_date


In [416]:
time_elements = response.xpath('/html/body/div/div/div[5]/ul/li/div/dl/dt/em[2]/text()')
start_date = time_elements[len(time_elements) - 1].extract().split(' ')[0]
start_date = start_date.split(" ")[0].split('.')
start_date

['2014', '12', '17']

In [417]:
len(time_elements)

2

In [395]:
cur_date = response.xpath('/html/body/div/div/div[5]/ul/li[{}]/div[2]/dl/dt/em[2]/text()'.format(5)).extract()[0].split(' ')[0].split('.')
cur_date


['2015', '04', '23']

In [390]:
rating_elements = response.xpath('/html/body/div/div/div[5]/ul/li[{}]/div[1]/em/text()'.format(5)).extract()[0]
int(rating_elements)

8

In [278]:
int(start_date[1])

4

In [281]:
limit_date = datetime.datetime(int(start_date[0]), int(start_date[1]), int(start_date[2])) + datetime.timedelta(weeks=1)
limit_date

datetime.datetime(2015, 4, 30, 0, 0)

In [302]:
check_limit = limit_date - datetime.datetime(2015, 5, 1)

In [304]:
check_limit.days < 0 

True

In [ ]:
# !scrapy startproject crawler

In [ ]:
# %%writefile crawler/crawler/items.py
# import scrapy


# class CrawlerItem(scrapy.Item):
#     expert_rating = scrapy.Field()
#     starring_actors = scrapy.Field()
#     running_time = scrapy.Field()
#     director = scrapy.Field()
    

In [ ]:
# %%writefile crawler/crawler/spiders/spider.py
# import scrapy

# from crawler.items import CrawlerItem

# class Spider(scrapy.Spider):
#     name = "MovieSpider"
#     allow_domain = ["movie.naver.com"]
#     start_urls = []
    
#     def parse(self, response):
#         total_df = pd.read_csv('code_added_df.csv').drop(columns=['Unnamed: 0'])
#         code_list = list(total_df['N_movie_code'])
        
#         for code in code_list:
#             url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code={}'.format(int(code))
#             yield scrapy.Request(url, callback=self.get_content)
            
#     def get_content(self, response):
#         item = CrawlerItem()
        
#         rate_number = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/div[1]/div[2]/div/a/div/em/text()').extract()
#         item['expert_rating'] = "".join(rate_number)
        
#         running_time = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[1]/p/span[3]/text()').extract()[0]
#         item['running_time'] = running_time
        
#         starring_actors = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[3]/p/a/text()').extract()
#         item['starring_actors'] = ", ".join(starring_actors)
        
#         director = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[2]/p/a/text()').extract()[0]
#         item['director'] = director
        
#         yield item
    

In [ ]:
# %%writefile run.sh
# cd crawler
# scrapy crawl GmarketSpider -o items.csv

In [ ]:
# 실행가능하도록 권한 바꾸기
# import os

# os.chmod("run.sh", 0o764)

In [ ]:
# 5. settings.py 수정
# !head -n 22 cat naver_article/naver_article/settings.py | tail -n 2

In [ ]:
# !sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' naver_article/naver_article/settings.py

In [ ]:
# !head -n 22 cat naver_article/naver_article/settings.py | tail -n 2

In [ ]:
# !./run.sh

In [ ]:
# csv파일로 저장된 데이터 확인

# items_df = pd.read_csv("naver_article/items.csv")
# items_df.tail()

In [184]:
df2 = pd.read_csv('code_added_df.csv').drop(columns=['Unnamed: 0'])
N_movie_code = list(df2['N_movie_code'])
len(N_movie_code)

928

In [188]:
from scrapy.http import TextResponse

In [209]:
url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code={}'.format(67786)
req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding="utf-8")

a = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[2]/p/a/text()').extract()[0]
a

'콜린 트레보로우'